In [114]:
import numpy as np
import keras
from keras.models import Sequential,Model
from keras.layers import Activation,MaxPooling2D,Dropout,concatenate,Input,Add
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from tensorflow.keras.models import load_model
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping,LearningRateScheduler,CSVLogger,LambdaCallback,TensorBoard
from keras import regularizers
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from keras.regularizers import l2
import seaborn as sns
import math
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/'
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.utils import plot_model
from sklearn.metrics import classification_report
%matplotlib inline


In [115]:
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
train_path = 'cats-and-dogs/train'
valid_path = 'cats-and-dogs/valid'
test_path  = 'cats-and-dogs/test'
save_path = "."
train_gen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    channel_shift_range=10.,
    fill_mode="nearest"
)

train_batches = train_gen.flow_from_directory(directory=train_path, target_size=(150,150),
    classes=['dog', 'cat'], batch_size=16)
valid_batches = ImageDataGenerator().flow_from_directory(directory=valid_path, target_size=(150,150), 
    classes=['dog', 'cat'], batch_size=16)
test_batches = ImageDataGenerator().flow_from_directory(directory=test_path, target_size=(150,150),
    classes=['dog', 'cat'], batch_size=16,shuffle=False)

Found 10514 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [116]:
lr_print_callback = LambdaCallback(
    on_epoch_begin=lambda epoch,logs: print("LearningRate of %e",model.optimizer.lr))

earlyStopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    verbose=0,
    mode='auto',
    baseline=None,
	patience=10,
    restore_best_weights=True
)

reduce_lr_loss = ReduceLROnPlateau(
    monitor='val_loss', 
	factor=0.1, 
    patience=7, 
	verbose=1, 
    min_delta=1e-4,
	mode='auto')
	
reduce_lr_loss = ReduceLROnPlateau(
    monitor='val_loss', 
	factor=0.1, 
    patience=7, 
	verbose=1, 
    min_delta=1e-4,
	mode='auto')

mcp_save       = ModelCheckpoint(
    'cat_dog_ckpt_inception.h5', 
	save_best_only=True, 
	monitor='val_loss', 
	mode='auto'
)

csv_log = CSVLogger('train.csv', separator=',', append=False)

In [117]:
# function for creating a projected inception module
def inception_module(layer_in, f1, f2_in, f2_out, f3_in, f3_out, f4_out):
	# 1x1 conv
	conv1 = Conv2D(f1, (1,1), padding='same', activation='relu',
                   kernel_initializer='glorot_uniform',
                   bias_initializer='zeros', kernel_regularizer=l2(0.0005))(layer_in)
	# 3x3 conv
	conv3 = Conv2D(f2_in, (1,1), padding='same', activation='relu',
                   kernel_initializer='glorot_uniform',
                   bias_initializer='zeros', kernel_regularizer=l2(0.0005))(layer_in)
                  
	conv3 = Conv2D(f2_out, (3,3), padding='same', activation='relu',
                   kernel_initializer='glorot_uniform',
                   bias_initializer='zeros', kernel_regularizer=l2(0.0005))(conv3)
	# 5x5 conv
	conv5 = Conv2D(f3_in, (1,1), padding='same', activation='relu',
                   kernel_initializer='glorot_uniform',
                   bias_initializer='zeros', kernel_regularizer=l2(0.0005))(layer_in)
	conv5 = Conv2D(f3_out, (5,5), padding='same', activation='relu',
                   kernel_initializer='glorot_uniform',
                   bias_initializer='zeros', kernel_regularizer=l2(0.0005))(conv5)
	# 3x3 max pooling
	pool = MaxPooling2D((3,3), strides=(1,1), padding='same')(layer_in)
	pool = Conv2D(f4_out, (1,1), padding='same', activation='relu')(pool)
	# concatenate filters, assumes filters/channels last
	layer_out = concatenate([conv1, conv3, conv5, pool], axis=-1)
	layer_out = Conv2D(32, (1,1), padding='same')(layer_out)
	layer_out = Add()([layer_in,layer_out])
	return layer_out



In [118]:
# define model input
visible    = Input(shape=(150, 150, 3))

conv_stem1 = Conv2D(32, (1,1), padding='valid', activation='relu',
                   kernel_initializer='glorot_uniform',
                   bias_initializer='zeros', kernel_regularizer=l2(0.0005))(visible)
bn_stem1   = BatchNormalization()(conv_stem1)
max_stem1  = MaxPooling2D(pool_size=(2, 2))(bn_stem1)                 
conv_stem2 = Conv2D(32, (1,1), padding='valid', activation='relu',
                   kernel_initializer='glorot_uniform',
                   bias_initializer='zeros', kernel_regularizer=l2(0.0005))(max_stem1)
max_stem1  = MaxPooling2D(pool_size=(2, 2))(conv_stem2) 
conv_stem3 = Conv2D(32, (1,1), padding='valid', activation='relu',
                   kernel_initializer='glorot_uniform',
                   bias_initializer='zeros', kernel_regularizer=l2(0.0005))(max_stem1)
bn_stem2   = BatchNormalization()(conv_stem3)
max_stem2  = MaxPooling2D(pool_size=(2, 2))(bn_stem2)
conv_stem4 = Conv2D(32, (1,1), padding='valid', activation='relu',
                   kernel_initializer='glorot_uniform',
                   bias_initializer='zeros', kernel_regularizer=l2(0.0005))(max_stem2)
# add inception block 1
layer = inception_module(conv_stem4, 64, 96, 128, 16, 32, 32)
#layer = Add()([conv_stem4,layer])
# add inception block 1
layer = inception_module(layer, 128, 128, 192, 64, 64, 64)
layer = inception_module(layer, 192, 192, 256, 128, 96, 96)
layer = inception_module(layer, 256, 256, 512, 192, 128, 128)
layer = MaxPooling2D(pool_size=(2, 2))(layer)
layer = Conv2D(256, (3,3), padding='valid', activation='relu',
                   kernel_initializer='glorot_uniform',
                   bias_initializer='zeros', kernel_regularizer=l2(0.0005))(layer)
layer = Dropout(0.5)(layer)
layer = Flatten()(layer)
layer = Dense(2, activation='sigmoid')(layer)
# create model
model = Model(inputs=visible, outputs=layer)
# summarize model
model.summary()
# plot model architecture
#plot_model(model, show_shapes=True, to_file='inception_module.png')
model.save('inception_experiment.h5')
model.compile(loss='binary_crossentropy',optimizer=Adam(lr=0.0001),metrics=['accuracy'])

Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_35 (InputLayer)           (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
conv2d_549 (Conv2D)             (None, 150, 150, 32) 128         input_35[0][0]                   
__________________________________________________________________________________________________
batch_normalization_63 (BatchNo (None, 150, 150, 32) 128         conv2d_549[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_177 (MaxPooling2D (None, 75, 75, 32)   0           batch_normalization_63[0][0]     
___________________________________________________________________________________________

In [119]:
history = model.fit_generator(generator=train_batches, steps_per_epoch=len(train_batches), 
                              validation_data=valid_batches, validation_steps=len(valid_batches), 
                              epochs=200, callbacks=[reduce_lr_loss],verbose=1) 
acc = history.history['accuracy'] 
val_acc = history.history['val_accuracy'] 
loss = history.history['loss'] 
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc') 
plt.title('Training and validation accuracy') 
plt.legend() 
plt.figure() 
plt.plot(epochs, loss, 'bo', label='Training loss') 
plt.plot(epochs, val_loss, 'b', label='Validation loss') 
plt.title('Training and validation loss') 
plt.savefig('accuracy.png')
plt.show()


Epoch 1/200
 13/658 [..............................] - ETA: 9:01 - loss: 1.7558 - accuracy: 0.4928

KeyboardInterrupt: 

In [ ]:
predictions = model.predict_generator(generator=test_batches, steps=len(test_batches), verbose=0)
pred_label = np.argmax(predictions,axis=1)
classes = np.argmax(predictions, axis=1)
cm = confusion_matrix(test_batches.labels,pred_label)
f,ax = plt.subplots(figsize=(4, 4))
sns.heatmap(cm, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()
plt.savefig('confusion_matrix.png')

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn import metrics
print(classification_report(test_batches.classes, pred_label,
	target_names=test_batches.class_indices.keys()))
tn, fp, fn, tp = cm.ravel()
accuracy = (tn + tp)/(tn + tp + fn +fp)
precision = precision_score(test_batches.labels, pred_label, average='binary')
recall = recall_score(test_batches.labels, pred_label,average='binary')
f1_score = f1_score(test_batches.labels, pred_label, average='binary')
score = metrics.accuracy_score(test_batches.labels, pred_label)
log_score = metrics.log_loss(pred_label, predictions)

In [ ]:
print("Precision ",precision*100)
print("Recall ",recall*100)
print("F1 Score ",recall*100)
print("Accuracy of the model",accuracy*100)
print("Accuracy score: {}".format(score))
print("Log loss score: {}".format(log_score))

In [ ]:
from sklearn.metrics import roc_curve, auc
def plot_roc(pred,y):
    fpr, tpr, _ = roc_curve(y, pred)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.show()
plot_roc(pred_label,test_batches.labels)